In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from continue_pretrain import TrainSciBERT
import torch
torch.cuda.is_available()

True

# Train

In [4]:
cls = TrainSciBERT()

In [5]:
import pandas as pd
df = pd.read_csv("/home/jiaruil5/anlp/model/data/scibert_nsp.csv")
a, b, l = df['sentence_a'].values.tolist(), df['sentence_b'].values.tolist(), df['label'].values.tolist()

In [6]:
inputs = cls.build_dataset(a, b, l)

In [ ]:
cls.train(
    inputs,
    "/home/jiaruil5/anlp/model/save/",
    **{
      'lr': 5e-5,
      'epochs': 5,
      'batch_size': 16,  
    },
)

  0%|          | 0/8547 [00:00<?, ?it/s]/home/jiaruil5/anlp/model/scibert/continue_pretrain.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1:  19%|█▉        | 1647/8547 [11:58<50:07,  2.29it/s, loss=0.117]  

# Test

In [3]:
cls = TrainSciBERT(model=False)

In [4]:
t_a, t_b, t_l = cls.build_test_corpus(
    "/home/jiaruil5/anlp/crawl/text/scipdf/json/",
    from_file="/home/jiaruil5/anlp/crawl/test_continue_pretrain.jsonl",
    # sample=10
)

28


In [22]:
t_a[0], t_b[0], t_l[0]

('While previous works have shown that unlabeled data in a target language can be used to improve crosslingual model performance, we propose a novel adversarial approach (AdvPicker) to better leverage such data and further improve results.',
 'We design an adversarial learning framework in which an encoder learns entity domain knowledge from labeled source-language data and better shared features are captured via adversarial training -where a discriminator selects less language-dependent target-language data via similarity to the source language.',
 0)

In [5]:
t_inputs = cls.build_dataset(t_a, t_b, t_l)

In [6]:
torch.cuda.empty_cache()
!nvidia-smi

Tue Oct 24 16:38:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:C1:00.0 Off |                  Off |
| 30%   30C    P8              28W / 300W |      5MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
torch.cuda.empty_cache()
from transformers import AutoTokenizer, BertForPreTraining
model = BertForPreTraining.from_pretrained('allenai/scibert_scivocab_uncased')
model.load_state_dict(torch.load("/home/jiaruil5/anlp/model/save/lr_5e-05_bs_16_epoch_0.pth"))
cls.eval(t_inputs, model)

  0%|          | 0/36 [00:00<?, ?it/s]/home/jiaruil5/anlp/model/scibert/continue_pretrain.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 36/36 [00:03<00:00,  9.46it/s]

0.38215051053298843


In [8]:
model.load_state_dict(torch.load("/home/jiaruil5/anlp/model/save/lr_5e-05_bs_16_epoch_1.pth"))
cls.eval(t_inputs, model)

100%|██████████| 36/36 [00:02<00:00, 13.21it/s]

0.3923173137009144


In [9]:
model.load_state_dict(torch.load("/home/jiaruil5/anlp/model/save/lr_5e-05_bs_16_epoch_2.pth"))
cls.eval(t_inputs, model)

100%|██████████| 36/36 [00:02<00:00, 13.19it/s]

0.4915901484588782


In [10]:
model.load_state_dict(torch.load("/home/jiaruil5/anlp/model/save/lr_5e-05_bs_16_epoch_3.pth"))
cls.eval(t_inputs, model)

100%|██████████| 36/36 [00:02<00:00, 13.17it/s]

0.5311935361888673


In [11]:
model.load_state_dict(torch.load("/home/jiaruil5/anlp/model/save/lr_5e-05_bs_16_epoch_4.pth"))
cls.eval(t_inputs, model)

100%|██████████| 36/36 [00:02<00:00, 13.15it/s]

0.6450290420196123
